In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import CategoricalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

# read csv
housing = pd.read_csv("../datasets/housing.csv")

# stratify the sample
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# create train and test sets
for train_index, test_index in split.split(housing, housing["income_cat"]):
    train_set = housing.iloc[train_index]
    test_set = housing.iloc[test_index]
for set_ in [train_set, test_set]:
    set_.drop("income_cat", axis=1, inplace=True)
    
# separate feature and label for training
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

# class: add attributes
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, roots_per_household, population_per_household]
        
# class: convert dataframe to numpy array
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
# class: convert category to one binary attribute
class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, encoding):
        self.encoding = encoding
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        df_X = pd.DataFrame({"A":X.ravel()})
        X_encoded = pd.factorize(df_X["A"])[0]
        encoder = OneHotEncoder()
        if self.encoding == "onehot-dense":
            return encoder.fit_transform(X_encoded.reshape(-1,1)).toarray()
        else:
            return encoder.fit_transform(X_encoded.reshape(-1,1))

# pipeline for numerical attributes
housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
num_pipeline = Pipeline([
    ("selector", DataFrameSelector(num_attribs)),
    ("imputer", Imputer(strategy="median")),
    ("attribs_added", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler())
])

# pipeline for category attributes
cat_attribs = ["ocean_proximity"]
cat_pipeline = Pipeline([
    ("selector", DataFrameSelector(cat_attribs)),
    ("cat_encoder", CategoricalEncoder(encoding="onehot-dense")),
])

# combine pipelines
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline)
])

# final prepare
housing_prepared = full_pipeline.fit_transform(housing)

# linear regression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)
housing_predictions = lin_reg.predict(housing_prepared)
lin_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions))

# decision tree
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)
housing_predictions = tree_reg.predict(housing_prepared)
tree_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions))

# random forest
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_rmse = np.sqrt(mean_squared_error(housing_labels, housing_predictions))

# cross-validation
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, 
                         scoring="neg_mean_squared_error", cv=2)
tree_rmse_scores = np.sqrt(-scores)
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, 
                         scoring="neg_mean_squared_error", cv=2)
lin_rmse_scores = np.sqrt(-scores)
scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=2)
forest_rmse_scores = np.sqrt(-scores)

# display scores
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
print(lin_rmse)
display_scores(tree_rmse_scores)
print(tree_rmse)
display_scores(lin_rmse_scores)
print(forest_rmse)
display_scores(forest_rmse_scores)

# save the parameters
#joblib.dump(lin_rmse, "ex_2_lin.pkl")
#lin_loaded = joblib.load("ex_2_lin.pkl")

# Grid search
param_grid = [
    {"n_estimators": [3, 10, 30], "max_features": [2, 4, 6, 8]},
    {"bootstrap": [False], "n_estimators": [3, 10], "max_features": [2, 3, 4]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring="neg_mean_squared_error")
grid_search.fit(housing_prepared, housing_labels)
print(grid_search.best_params_)
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
feature_importances = grid_search.best_estimator_.feature_importances_
print(feature_importances)

# test on train set
final_model = grid_search.best_estimator_
X_test = test_set.drop("median_house_value", axis=1)
y_test = test_set["median_house_value"].copy()
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)
final_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
print("Final RMSE on test sample:", final_rmse)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


68632.1192054
Scores: [ 74000.0873989   73083.30586839]
Mean: 73541.6966336
Standard deviation: 458.390765259
0.0
Scores: [ 69801.88130956  69171.62740611]
Mean: 69486.7543578
Standard deviation: 315.126951724
22150.1314683
Scores: [ 55042.66295174  54812.0958612 ]
Mean: 54927.3794065
Standard deviation: 115.283545272
{'max_features': 8, 'n_estimators': 30}
64043.7245878 {'max_features': 2, 'n_estimators': 3}
55470.7200036 {'max_features': 2, 'n_estimators': 10}
52709.8258573 {'max_features': 2, 'n_estimators': 30}
59194.0719041 {'max_features': 4, 'n_estimators': 3}
52857.91141 {'max_features': 4, 'n_estimators': 10}
50609.9068248 {'max_features': 4, 'n_estimators': 30}
59039.3511548 {'max_features': 6, 'n_estimators': 3}
52146.0701181 {'max_features': 6, 'n_estimators': 10}
50302.7433707 {'max_features': 6, 'n_estimators': 30}
58987.4826734 {'max_features': 8, 'n_estimators': 3}
52007.2811774 {'max_features': 8, 'n_estimators': 10}
50092.3140313 {'max_features': 8, 'n_estimators': 30